In [14]:
import requests

import pandas as pd

from datetime import datetime

from pprint import pprint

from datetime import datetime, timedelta

import plotly.express as px
import plotly.colors as pc

In [15]:
lang    =  input('¿En español (es) o en inglés (en)?') 

restaDia = float(input("¿Cuantos días atrás?: ______________"))
input_año = int(input("¿Qué año?: ______________"))

In [16]:
headers = {
    "Accept": "application/json",
    "Content_Type": "application/json",
    "Host": "apidatos.ree.es"
}

In [17]:
def demanda_datos(lang, restaDia, input_año):
     
    now = datetime.now()
    ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')


    hoy = now.replace(year = input_año).strftime('%Y-%m-%d')

    query = f"start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day"

    endpoint = f"https://apidatos.ree.es/{lang}/datos/demanda/evolucion?{query}"
    response = requests.get(url = endpoint, headers = headers)
    data = response.json()

    datetime_lista = list()
    value_lista = list()
    percentage_lista = list()



    for value in data['included']:
        for content in value['attributes']['values']:
                
                porcentaje = content['percentage']
                valor = content['value']
                
                fecha = content ['datetime']
                fecha = pd.to_datetime(fecha)
                fecha = fecha.strftime("%d/%m/%Y")

                datetime_lista.append(fecha)
                value_lista.append(valor)
                percentage_lista.append(porcentaje)


    df_demanda = pd.DataFrame()
    df_demanda['Fecha actualización'] = datetime_lista
    df_demanda['Energia_consumida'] = value_lista

    return df_demanda

df_demanda = demanda_datos(lang, restaDia, input_año)

df_demanda


,Fecha actualización,Energia_consumida
0,03/07/2024,716572.221
1,04/07/2024,737273.306
2,05/07/2024,738941.786
3,06/07/2024,650171.313
4,07/07/2024,588034.182
...,...,...
96,07/10/2024,665564.644
97,08/10/2024,682877.223
98,09/10/2024,678971.738
99,10/10/2024,669069.836


In [18]:
df_demanda.to_csv('Demanda_evolucion.csv')

In [19]:
colores_personalizados = px.colors.qualitative.Plotly + px.colors.qualitative.Pastel + px.colors.qualitative.Set1
colores_personalizados = colores_personalizados[:30]

In [20]:
fig_demanda = px.line(data_frame = df_demanda,
        x = 'Fecha actualización',
        y = 'Energia_consumida'
)

fig_demanda.update_layout(title = 'Evolución de demanda energética diaría')

fig_demanda.show()